In [1]:
import pyodbc
import pandas as pd
import numpy as np

C:\Users\krishnateja\AppData\Local\Temp\ipykernel_16152\2275304727.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("./Dataset/SupplyChainFinal.csv", encoding='latin-1')
df.head()

,order_id,order_date,delivery_date,actual_shipping_days,expected_shipping_days,delivery_status,is_late_delivery,warehouse_street,warehouse_city,warehouse_state,...,customer_id,customer_fname,customer_lname,customer_segment,cost_price,profit_amount,product_price,order_item_discount,price_after_discount,order_product_quantity
0,42,2015-01-01,2015-01-03,2,1,Late delivery,1,2328 Rustic Elk Abbey,Endicott,NY,...,9776,Diane,Davis,Consumer,39.990002,3.750000,39.990002,6.800000,33.189999,1.0
1,16,2015-01-01,2015-01-03,2,4,Advance shipping,0,9243 Old Gardens,Caguas,PR,...,7276,Pamela,Smith,Consumer,299.950012,24.000000,59.990002,59.990002,239.960007,5.0
2,57,2015-01-01,2015-01-04,3,4,Advance shipping,0,1987 Grand Concession,Del Rio,TX,...,7073,Joan,Smith,Consumer,129.990005,26.320000,129.990005,32.500000,97.489998,1.0
3,62,2015-01-01,2015-01-04,3,4,Advance shipping,0,2122 Green By-pass,Caguas,PR,...,9111,Mary,Smith,Corporate,50.000000,14.200000,50.000000,8.000000,42.000000,1.0
4,15,2015-01-01,2015-01-07,6,4,Late delivery,1,3544 Fallen Mount,Memphis,TN,...,2568,Maria,Smith,Consumer,50.000000,-61.880001,50.000000,12.500000,37.500000,1.0


In [3]:
df.shape

(180495, 33)

In [4]:
# remove columns delivery_status, is_late_delivery, cost_price, price_after_discount
df.drop(['delivery_status', 'is_late_delivery', 'cost_price', 'price_after_discount'], axis=1, inplace=True)

In [5]:
df.shape

(180502, 29)

In [5]:
conn = pyodbc.connect(r'Driver=SQL Server;Server=UITS-DG3MPKS3;Database=MIS587_Project;Trusted_Connection=yes;')
cursor = conn.cursor()

In [6]:
# create a table in called DataCoOLTP with the columns in the dataframe

cursor.execute("""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'DataCoOLTP')
BEGIN
CREATE TABLE dbo.DataCoOLTP (
    order_id int,
    order_date date,
    delivery_date date,
    actual_shipping_days int,
    expected_shipping_days int,
    warehouse_street varchar(255),
    warehouse_city varchar(50),
    warehouse_state varchar(50),
    warehouse_country varchar(50),
    order_city varchar(50),
    order_state varchar(50),
    order_country varchar(50),
    order_region varchar(50),
    market varchar(50),
    latitude decimal(9, 6),
    longitude decimal(9, 6),
    shipping_mode varchar(50),
    product_id int,
    product_name varchar(255),
    category_name varchar(50),
    department_name varchar(50),
    customer_id int,
    customer_fname varchar(100),
    customer_lname varchar(100),
    customer_segment varchar(50),
    profit_amount decimal(10, 2),
    product_price decimal(10, 2),
    order_item_discount decimal(10, 2),
    order_product_quantity int
)
END
""")

conn.commit()

In [7]:
insert_stmt = '''
INSERT INTO dbo.DataCoOLTP (
    order_id, order_date, delivery_date, actual_shipping_days, expected_shipping_days, warehouse_street, 
    warehouse_city, warehouse_state, warehouse_country, order_city, order_state, order_country, order_region, market, 
    latitude, longitude, shipping_mode, product_id, product_name, category_name, department_name, customer_id, customer_fname, 
    customer_lname, customer_segment, profit_amount, product_price, order_item_discount, order_product_quantity

) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

In [9]:
# Prepare and execute the SQL statements
for index, row in df.iterrows():
    values = tuple(row[c] for c in df.columns)
    cursor.execute(insert_stmt, values)

# Commit the transaction
conn.commit()

# Close the connection
cursor.close()
conn.close()

In [6]:
# initiate a cursoe names dw_cursor
dw_conn = pyodbc.connect(r'Driver=SQL Server;Server=UITS-DG3MPKS3;Database=MIS587_Project;Trusted_Connection=yes;')
dw_cursor = dw_conn.cursor()


In [14]:
# Orders Dimension
dw_cursor.execute("""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'ordersDim')
BEGIN
CREATE TABLE dbo.ordersDim (
    order_skey int identity(1,1) primary key,
        order_id INT,
        order_date DATE,
        delivery_date DATE,
        actual_shipping_days int,
        expected_shipping_days int,
        delivery_status varchar(50),
        is_delivery_late bit,
        order_city varchar(50),
        order_country varchar(50),
        latitude float,
        longitude float,
        market varchar(50),
        order_region varchar(50),
        order_state varchar(50),
        warehouse_city VARCHAR(50),
        warehouse_country VARCHAR(50),
        warehouse_street VARCHAR(255),
        warehouse_state VARCHAR(50),
        shipping_mode varchar(50) 
)
END
""")

dw_conn.commit()

In [39]:
# Prouct Dimension
dw_cursor.execute("""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'productDim')
BEGIN
CREATE TABLE dbo.productDim (
    product_skey INT IDENTITY(1,1) PRIMARY KEY,
    product_id INT,
    category_name VARCHAR(50),
    department_name VARCHAR(50),
    product_name VARCHAR(255)
)
END
""")

dw_conn.commit()

In [41]:
# Customer Dimension
dw_cursor.execute("""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'customerDim')
BEGIN
CREATE TABLE dbo.customerDim (
    customer_skey INT IDENTITY(1,1) PRIMARY KEY,
    customer_id INT,
    customer_fname VARCHAR(100),
    customer_lname VARCHAR(100),
    customer_name VARCHAR(200),
    customer_segment VARCHAR(50)
)
END
""")

dw_conn.commit()

In [42]:
# Date Dimension
dw_cursor.execute("""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'dateDim')
BEGIN
CREATE TABLE dbo.dateDim (
    date_skey INT IDENTITY(1,1) PRIMARY KEY,
    date DATE,
    day INT,
    month INT,
    year INT
)
END
""")

dw_conn.commit()

In [15]:
# OrderFact Fact Table
dw_cursor.execute("""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME = 'ordersFact')
BEGIN
CREATE TABLE dbo.ordersFact (
    order_skey INT,
    product_skey INT,
    customer_skey INT,
    order_date_skey INT,
    order_id INT,
    product_id INT,
    product_price DECIMAL(10, 2),
    order_product_quantity INT,
    cost_price DECIMAL(10, 2),
    order_item_discount DECIMAL(10, 2),
    price_after_discount DECIMAL(10, 2),
    profit_amount DECIMAL(10, 2)
)
END
""")

dw_conn.commit()